In [3]:
import time

In [9]:
a = time.time()
a

1676964862.2225564

In [8]:
time.clock_gettime(time.asctimea)

TypeError: integer argument expected, got float

In [11]:
time.asctime((a))

TypeError: Tuple or struct_time argument required

In [18]:
time.strptime(time.asctime(time.gmtime(a)))

time.struct_time(tm_year=2023, tm_mon=2, tm_mday=21, tm_hour=7, tm_min=34, tm_sec=22, tm_wday=1, tm_yday=52, tm_isdst=-1)

In [26]:
time.time()

1676965127.976975